In [313]:
%matplotlib
import geopandas as gpd
import glob
from itertools import tee
import networkx as nx
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import json
import os
from collections import defaultdict
from shapely.geometry import Point, Polygon, LineString

Roads = gpd.read_file("/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_City_Version3/N-N_Roads_newline.shp")

def pairwise(line_coords):
    a,b = tee(line_coords, 2)
    next(b, None)
    return zip(a,b) #return (p1,p2), (p2,p3)...

def create_g(Geo_dataFrame):
    G = nx.DiGraph()
    for line in Geo_dataFrame.geometry:
       seq = pairwise(line.coords)
       for p1, p2 in seq:
           G.add_edge(p1, p2)
    return G

#Euclidean distance
const = np.sqrt(2)
def dist(n0, n1):
    x1, y1 = n0
    x2, y2 = n1
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def closest_node(sub_graph, start, end): 
    start_delta = float("inf")
    end_delta = float("inf")
    for n in sub_graph.nodes():
        s_dist = dist(start, n)
        e_dist = dist(end, n) 
        if s_dist < start_delta:
            nn_start = n
            start_delta = s_dist
        if e_dist < end_delta:
            nn_end = n
            end_delta = e_dist
    return (nn_start, nn_end)

def add_attr(sub_graph):
    for n0, n1 in sub_graph.edges_iter():
        distance = dist(n0, n1)
        sub_graph[n0][n1]["dist"] = distance

def shortest_dist(sub_graph, nn_start, nn_end):
    shortest_distance = nx.shortest_path_length(sub_graph, source= nn_start ,target= nn_end, weight= "dist")
    return shortest_distance
def shortest_path(sub_graph, nn_start, nn_end):
    shortest_path = nx.shortest_path(sub_graph, source= nn_start, target= nn_end, weight= "dist")
    return shortest_path
def calculate_distance(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    shortest_distance = shortest_dist(graph, nn_start, nn_end)
    return shortest_distance

def calculate_path(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    optimal_path = shortest_path(graph, nn_start, nn_end)
    return optimal_path

#convert road into graph
G = create_g(Roads)
sg = list(nx.connected_component_subgraphs(G.to_undirected()))[0]
add_attr(sg) #add distance attribute

Using matplotlib backend: MacOSX


In [314]:
%matplotlib
import geopandas as gpd
import glob
from itertools import tee
import networkx as nx
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import json
import os
from collections import defaultdict
from shapely.geometry import Point, Polygon, LineString

Roads = gpd.read_file("/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_City_Version3/N-N_Roads_newline.shp")
Buildings = gpd.read_file("/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/N-N_Buildings.shp")
# Shuttle Route in North Campus
Route1 = gpd.read_file('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/NC-inner-route.shp') 
G1 = create_g(Route1)
sg1 = list(nx.connected_component_subgraphs(G1.to_undirected()))[0]
add_attr(sg1)
# Shuttle Route from North Campus to NCRC
Route2 = gpd.read_file('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/NC-NCRC_fixedroute.shp')
G2 = create_g(Route2)
sg2 = list(nx.connected_component_subgraphs(G2.to_undirected()))[0]
add_attr(sg2)
# Shuttle Route from NCRC to North Campus
Route3 = gpd.read_file('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/AA_city_NAD83/NCRC-NC_fixedroute.shp')
G3 = create_g(Route3)
sg3 = list(nx.connected_component_subgraphs(G3.to_undirected()))[0]
add_attr(sg3)
# bus_stops
Lurie = (276278.1061585448, 4685419.189922427)
stop_NCRC = (276980.0661116603, 4686804.277913463)

# buildings "Objectid_1" not equal to 33,61,79
building = Buildings[(Buildings.OBJECTID_1!=33)&(Buildings.OBJECTID_1!=61)&(Buildings.OBJECTID_1!=79)]
lat_line = 290600 # line between North Campus and NCRC, 100000
building_NC = building[building.Latitude<lat_line] # building in North Campus
building_NCRC = building[building.Latitude>lat_line] #building in NCRC

Using matplotlib backend: MacOSX


In [315]:
# NorthCampus-NCRC trip sample
trip1 = pd.read_csv('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/Trip/NC-NCRC_trip/NorthCampus-UMTRI_sample.csv')
# original data
import datetime
O_index = building_NC[building_NC.OBJECTID_1==trip1.loc[0,'O_id']].index[0]
D_index = building_NCRC[building_NCRC.OBJECTID_1==trip1.loc[0,'D_id']].index[0]
trip1.loc[0,'O_lon'] = building_NC.geometry[O_index].coords[0][0]
trip1.loc[0,'O_lat'] = building_NC.geometry[O_index].coords[0][1]
trip1.loc[0,'start_t'] = datetime.datetime(2017,7,14,16,17,9)
trip1.loc[0,'D_lon'] = building_NCRC.geometry[D_index].coords[0][0]
trip1.loc[0,'D_lat'] = building_NCRC.geometry[D_index].coords[0][1]
O = trip1.loc[0,'O_lon'],trip1.loc[0,'O_lat']
D = trip1.loc[0,'D_lon'],trip1.loc[0,'D_lat']
trip1.loc[0,'drivetrip_sec'] = 107.0
trip1.loc[0,'bus_t'] = trip1.loc[0,'start_t'] + datetime.timedelta(0,int(const*dist(O, Lurie)/1.5))
trip1.loc[0,'shuttletrip_sec'] = int(calculate_distance(Lurie,stop_NCRC,sg2)/11+const*(dist(O,Lurie)+dist(D,stop_NCRC))/1.5)

In [333]:
# random data----------------------------------------------
# random 15 trips----------------------------
import random
NorthCampus_list = building_NC.OBJECTID_1.tolist()
NCRC_list = building_NCRC.OBJECTID_1.tolist()
# NorthCampus to NCRC
O_list = []
D_list = []
i = 0
while i < 15:
    O_list.append(random.choice(NorthCampus_list))
    D_list.append(random.choice(NCRC_list))
    i = i + 1
# NCRC to North Campus
O_list1 = []
D_list1 = []
i = 0
while i < 16:
    O_list1.append(random.choice(NCRC_list))
    D_list1.append(random.choice(NorthCampus_list))
    i = i + 1
# random parking time---------------------------------
import scipy.stats
lower = 0
upper = 300
mu = 90         # average parking time, unit of sec
sigma = 70      # variance parking time
N = 15          # 15 trips
# NorthCampus to NCRC
parkingtime_list = scipy.stats.truncnorm.rvs(
          (lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=N).tolist()
# NCRC to NorthCampus
N1 = 16
parkingtime_list1 = scipy.stats.truncnorm.rvs(
          (lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=N1).tolist()
# random start time---------------------------------
import time
starttime_list = []
starttime_list1 = []
t_upper = datetime.datetime(2017,7,14,15,45,0)
t_lower = datetime.datetime(2017,7,14,16,45,0)
unixtime_upper = time.mktime(t_upper.timetuple())
unixtime_lower = time.mktime(t_lower.timetuple())
# NorthCampus to NCRC
i = 0
while i < 15:
    tmp = random.randrange(unixtime_upper, unixtime_lower)
    starttime_list.append(datetime.datetime.fromtimestamp(tmp))
    i = i + 1
# NCRC to NorthCampus
i = 0
while i < 16:
    tmp = random.randrange(unixtime_upper, unixtime_lower)
    starttime_list1.append(datetime.datetime.fromtimestamp(tmp))
    i = i + 1

In [334]:
shuttle_t1 = calculate_distance(Lurie,stop_NCRC,sg2)/11
shuttle_t2 = calculate_distance(stop_NCRC,Lurie,sg3)/11
for i in trip1[1:].index:
    trip1.loc[i,'O_id'] = O_list[i-1]
    trip1.loc[i,'D_id'] = D_list[i-1]
    O_index = building_NC[building_NC.OBJECTID_1==trip1.loc[i,'O_id']].index[0]
    D_index = building_NCRC[building_NCRC.OBJECTID_1==trip1.loc[i,'D_id']].index[0]
    trip1.loc[i,'O_lon'] = building_NC.geometry[O_index].coords[0][0]
    trip1.loc[i,'O_lat'] = building_NC.geometry[O_index].coords[0][1]
    trip1.loc[i,'D_lon'] = building_NCRC.geometry[D_index].coords[0][0]
    trip1.loc[i,'D_lat'] = building_NCRC.geometry[D_index].coords[0][1]
    trip1.loc[i,'start_t'] = starttime_list[i-1]
    # calulate drivetrip time
    O = trip1.loc[i,'O_lon'],trip1.loc[i,'O_lat']
    D = trip1.loc[i,'D_lon'],trip1.loc[i,'D_lat']
    t_delta = int(calculate_distance(O, D, sg)/11 + parkingtime_list[i-1])
    trip1.loc[i,'drivetrip_sec'] = t_delta
    # calculate shuttletrip time
    trip1.loc[i,'bus_t'] = trip1.loc[i,'start_t'] + datetime.timedelta(0,int(const*dist(O, Lurie)/1.5))
    trip1.loc[i,'shuttletrip_sec'] = int(shuttle_t1+const*(dist(O,Lurie)+dist(D,stop_NCRC))/1.5)

In [339]:
# NCRC-NorthCampus trip sample
trip2 = pd.read_csv('/Users/mac/Google 云端硬盘/M_City_Shuttle/AA_Map/Trip/NC-NCRC_trip/UMTRI-NorthCampus_sample.csv')
for i in trip2.index:
    trip2.loc[i,'O_id'] = O_list1[i]
    trip2.loc[i,'D_id'] = D_list1[i]
    O_index = building_NCRC[building_NCRC.OBJECTID_1==trip2.loc[i,'O_id']].index[0]
    D_index = building_NC[building_NC.OBJECTID_1==trip2.loc[i,'D_id']].index[0]
    trip2.loc[i,'O_lon'] = building_NCRC.geometry[O_index].coords[0][0]
    trip2.loc[i,'O_lat'] = building_NCRC.geometry[O_index].coords[0][1]
    trip2.loc[i,'D_lon'] = building_NC.geometry[D_index].coords[0][0]
    trip2.loc[i,'D_lat'] = building_NC.geometry[D_index].coords[0][1]
    trip2.loc[i,'start_t'] = starttime_list1[i]
    # calulate drivetrip time
    O = trip2.loc[i,'O_lon'],trip2.loc[i,'O_lat']
    D = trip2.loc[i,'D_lon'],trip2.loc[i,'D_lat']
    t_delta = int(calculate_distance(O, D, sg)/11 + parkingtime_list1[i])
    trip2.loc[i,'drivetrip_sec'] = t_delta
    # calculate shuttletrip time
    trip2.loc[i,'bus_t'] = trip2.loc[i,'start_t'] + datetime.timedelta(0,int(const*dist(O, stop_NCRC)/1.5))
    trip2.loc[i,'shuttletrip_sec'] = int(shuttle_t2+const*(dist(O,stop_NCRC)+dist(D,Lurie))/1.5)

In [420]:
# shuttle scheduling---------------------------------------------------------------------
T1 = datetime.timedelta(0,120)    # shuttle waitingtime at Lurie, unit of sec
T2 = datetime.timedelta(0,120)    # shuttle waitingtime at stop_NCRC, unit of sec
t1_dt = datetime.timedelta(0,int(shuttle_t1))    # shuttle drivetime from NorthCampus to NCRC
t2_dt = datetime.timedelta(0,int(shuttle_t2))    # shuttle drivetime from NCRC to NorthCampus
patience_t = datetime.timedelta(0,60)            # patience of people waiting at the bus-stop
shuttle1_start = datetime.datetime(2017,7,14,15,49,0)      # start from Lurie
shuttle2_start = shuttle1_start + 0.5*(T1 + t1_dt + T2 + t2_dt)
Lurie_time = [];list1 = []      
stop_NCRC_time = [];list2 = []
tmp1 = shuttle1_start
while tmp1 < t_lower + T1 + t1_dt + T2 + t2_dt:
    list1.append(tmp1)
    list2.append(tmp1+T1+t1_dt)
    tmp1 = tmp1 + T1 + t1_dt + T2 + t2_dt
tmp2 = shuttle2_start
while tmp2 < t_lower + T1 + t1_dt + T2 + t2_dt:
    list1.append(tmp2)
    list2.append(tmp2+T1+t1_dt)
    tmp2 = tmp2 + T1 + t1_dt + T2 + t2_dt
Lurie_time = sorted(list1)
stop_NCRC_time = sorted(list2)

In [421]:
a = 0
b = 0 
for i in trip1.index:
    t2 = [x for x in Lurie_time if x >= trip1.loc[i,'bus_t']][0]
    t1 = Lurie_time[Lurie_time.index(t2)-1]
    if (trip1.loc[i,'bus_t'] <= t1+T1)or(trip1.loc[i,'bus_t']+patience_t >= t2):a=a+1
    t4 = [x for x in stop_NCRC_time if x >= trip2.loc[i,'bus_t']][0]
    t3 = stop_NCRC_time[stop_NCRC_time.index(t4)-1]
    if (trip2.loc[i,'bus_t'] <= t3+T2)or(trip2.loc[i,'bus_t']+patience_t >= t4):b=b+1

In [422]:
(a,b)

(14, 11)

In [413]:
Lurie_time

[datetime.datetime(2017, 7, 14, 15, 49),
 datetime.datetime(2017, 7, 14, 15, 53, 35, 500000),
 datetime.datetime(2017, 7, 14, 15, 58, 11),
 datetime.datetime(2017, 7, 14, 16, 2, 46, 500000),
 datetime.datetime(2017, 7, 14, 16, 7, 22),
 datetime.datetime(2017, 7, 14, 16, 11, 57, 500000),
 datetime.datetime(2017, 7, 14, 16, 16, 33),
 datetime.datetime(2017, 7, 14, 16, 21, 8, 500000),
 datetime.datetime(2017, 7, 14, 16, 25, 44),
 datetime.datetime(2017, 7, 14, 16, 30, 19, 500000),
 datetime.datetime(2017, 7, 14, 16, 34, 55),
 datetime.datetime(2017, 7, 14, 16, 39, 30, 500000),
 datetime.datetime(2017, 7, 14, 16, 44, 6),
 datetime.datetime(2017, 7, 14, 16, 48, 41, 500000),
 datetime.datetime(2017, 7, 14, 16, 53, 17)]

In [344]:
trip2.sort_values('bus_t')

,trip_id,O_id,O_lon,O_lat,D_id,D_lon,D_lat,start_t,drivetrip_sec,bus_t,shuttletrip_sec
13,13,108,276884.647214,4.686654e+06,43,276279.198266,4.685612e+06,2017-07-14 15:46:54,384,2017-07-14 15:49:41,511
12,12,103,276810.027065,4.686719e+06,57,276114.852293,4.685676e+06,2017-07-14 15:51:00,257,2017-07-14 15:53:59,628
5,5,100,276805.051028,4.686584e+06,34,275981.996859,4.685516e+06,2017-07-14 15:51:02,394,2017-07-14 15:55:27,721
1,1,91,277174.169121,4.686422e+06,34,275981.996859,4.685516e+06,2017-07-14 15:50:56,262,2017-07-14 15:57:40,860
6,6,99,276922.731015,4.686548e+06,2,276390.169234,4.685487e+06,2017-07-14 15:53:50,263,2017-07-14 15:57:57,533
8,8,100,276805.051028,4.686584e+06,32,276384.305645,4.685378e+06,2017-07-14 15:59:06,245,2017-07-14 16:03:31,535
0,0,89,277244.829408,4.686603e+06,10,276209.442954,4.685509e+06,2017-07-14 16:00:11,252,2017-07-14 16:05:24,582
9,9,77,276751.080283,4.686628e+06,11,276075.789472,4.685767e+06,2017-07-14 16:19:37,203,2017-07-14 16:24:09,813
10,10,107,277021.124495,4.686704e+06,18,275968.895852,4.685763e+06,2017-07-14 16:24:55,327,2017-07-14 16:26:37,700
15,15,99,276922.731015,4.686548e+06,50,276031.919749,4.685650e+06,2017-07-14 16:25:01,358,2017-07-14 16:29:08,728


In [345]:
trip1.sort_values('bus_t')

,trip_id,O_id,O_lon,O_lat,D_id,D_lon,D_lat,start_t,drivetrip_sec,bus_t,shuttletrip_sec
14,14,37,276073.297207,4.685876e+06,91,277174.169121,4.686422e+06,2017-07-14 15:47:05,288.0,2017-07-14 15:54:57,1025
15,15,47,276533.619245,4.685900e+06,91,277174.169121,4.686422e+06,2017-07-14 15:46:28,156.0,2017-07-14 15:55:01,1067
10,10,48,276843.632873,4.685788e+06,107,277021.124495,4.686704e+06,2017-07-14 15:47:07,212.0,2017-07-14 15:57:43,888
3,3,32,276384.305645,4.685378e+06,74,276994.519815,4.686535e+06,2017-07-14 15:58:06,218.0,2017-07-14 15:59:53,510
6,6,45,276533.567517,4.685939e+06,76,276980.066112,4.686804e+06,2017-07-14 15:56:48,296.0,2017-07-14 16:05:54,696
1,1,15,276660.262929,4.685911e+06,102,276821.683293,4.686478e+06,2017-07-14 16:03:22,257.0,2017-07-14 16:13:08,1078
5,5,113,276469.656326,4.686025e+06,68,277275.509774,4.686492e+06,2017-07-14 16:15:02,185.0,2017-07-14 16:25:00,1153
0,0,3,276661.804924,4.685872e+06,91,277174.169121,4.686422e+06,2017-07-14 16:17:09,107.0,2017-07-14 16:26:28,1113
9,9,13,276192.119162,4.685649e+06,100,276805.051028,4.686584e+06,2017-07-14 16:31:39,252.0,2017-07-14 16:35:30,646
7,7,57,276114.852293,4.685676e+06,73,276940.255590,4.686480e+06,2017-07-14 16:31:15,343.0,2017-07-14 16:36:02,744
